In [16]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%config InlineBackend.figure_format = 'retina'

import numpy as np
np.set_printoptions(precision=2, linewidth=120, suppress=True, edgeitems=4)

import pandas as pd
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 50)
#pd.set_option('precision', 5)

In [104]:
import tensorflow
import string

from nltk.corpus import stopwords as sw
from nltk.corpus import wordnet as wn
from nltk import wordpunct_tokenize
from nltk import WordNetLemmatizer
from nltk import sent_tokenize
from nltk import pos_tag

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from gensim.models import Word2Vec, Doc2Vec

In [83]:
train = pd.read_csv('train.csv',index_col='id')
test = pd.read_csv('test.csv',index_col='id')

In [84]:
train.head()
train.shape
#train.iloc[:, np.r_[1:6,6]] # Select multiple ranges of columns

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,,
22256635,"Nonsense? kiss off, geek. what I said is true...",1,0,0,0,0,0
27450690,"""\n\n Please do not vandalize pages, as you di...",0,0,0,0,0,0
54037174,"""\n\n """"Points of interest"""" \n\nI removed the...",0,0,0,0,0,0
77493077,Asking some his nationality is a Racial offenc...,0,0,0,0,0,0
79357270,The reader here is not going by my say so for ...,0,0,0,0,0,0


(95851, 7)

In [85]:
test.head()
test.shape

,comment_text
id,
6044863,==Orphaned non-free media (Image:41cD1jboEvL. ...
6102620,::Kentuckiana is colloquial. Even though the ...
14563293,"Hello fellow Wikipedians,\nI have just modifie..."
21086297,"AKC Suspensions \nThe Morning Call - Feb 24, 2..."
22982444,== [WIKI_LINK: Talk:Celts] ==


(226998, 1)

In [86]:
## We want to remove any newline characters

train = train.replace('\n','', regex=True)
train = train.replace('\r','', regex=True)

test = test.replace('\n','', regex=True)
test = test.replace('\r','', regex=True)

In [87]:
## Find out how many times the classes occur
train.iloc[:,1:].sum()

toxic            9237
severe_toxic      965
obscene          5109
threat            305
insult           4765
identity_hate     814
dtype: int64

In [88]:
negs = train[train.iloc[:,1:7].sum(axis=1) == 0] # All rows where there is not a single positive class
negs.head()
negs.shape

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,,
27450690,""" Please do not vandalize pages, as you did wi...",0,0,0,0,0,0
54037174,""" """"Points of interest"""" I removed the """"point...",0,0,0,0,0,0
77493077,Asking some his nationality is a Racial offenc...,0,0,0,0,0,0
79357270,The reader here is not going by my say so for ...,0,0,0,0,0,0
82428052,Fried chickens Is dat sum fried chickens?,0,0,0,0,0,0


(86061, 7)

In [89]:
pos = train[train.iloc[:,1:7].sum(axis=1) > 0] # All rows where there is at least one class marked positive
pos.head()
pos.shape

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,,
22256635,"Nonsense? kiss off, geek. what I said is true...",1,0,0,0,0,0
225701312,Why hasn't Alitalia been removed rom the allia...,1,0,0,0,0,0
293668009,"""The Graceful Slick....Is non other than an un...",1,0,0,0,0,0
341549388,""" Stupid? As soon as I saw the phrase """"metal ...",1,0,0,0,0,0
345843351,"""Ban one side of an argument by a bullshit naz...",1,0,1,0,1,0


(9790, 7)

In [90]:
corpus = train['comment_text']

In [91]:
corpus.head()

id
22256635    Nonsense?  kiss off, geek. what I said is true...
27450690    " Please do not vandalize pages, as you did wi...
54037174    " ""Points of interest"" I removed the ""point...
77493077    Asking some his nationality is a Racial offenc...
79357270    The reader here is not going by my say so for ...
Name: comment_text, dtype: object

In [100]:
corpus.comment_text

AttributeError: 'Series' object has no attribute 'comment_text'

In [102]:
## Create one large string out of the corpus

train.comment_text.cat(sep='...')

AttributeError: Can only use .cat accessor with a 'category' dtype

In [93]:
# convert to lower case
corpus_raw = corpus.lower()

AttributeError: 'Series' object has no attribute 'lower'

https://towardsdatascience.com/learn-word2vec-by-implementing-it-in-tensorflow-45641adaf2ac

In [ ]:
## Create dictionary

words = []

for word in corpus_raw.split():
    if word != '.': # because we don't want to treat . as a word
        words.append(word)

words = set(words) # so that all duplicate words are removed

word2int = {}
int2word = {}

vocab_size = len(words) # gives the total number of unique words

for i,word in enumerate(words):
    word2int[word] = i
    int2word[i] = word

In [ ]:
## Get a list of sentences

# raw sentences is a list of sentences.
raw_sentences = corpus_raw.split('.')

sentences = []

for sentence in raw_sentences:
    sentences.append(sentence.split())

In [ ]:
## Generate training data

data = []

WINDOW_SIZE = 2

for sentence in sentences:
    for word_index, word in enumerate(sentence):
        for nb_word in sentence[max(word_index - WINDOW_SIZE, 0) : min(word_index + WINDOW_SIZE, len(sentence)) + 1] : 
            if nb_word != word:
                data.append([word, nb_word])

In [ ]:
## One-hot encoding

# function to convert numbers to one hot vectors
def to_one_hot(data_point_index, vocab_size):
    temp = np.zeros(vocab_size)
    temp[data_point_index] = 1
    return temp

x_train = [] # input word
y_train = [] # output word

for data_word in data:
    x_train.append(to_one_hot(word2int[ data_word[0] ], vocab_size))
    y_train.append(to_one_hot(word2int[ data_word[1] ], vocab_size))

# convert them to numpy arrays
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)

https://de.dariah.eu/tatom/preprocessing.html

In [106]:
corpus

id
22256635        Nonsense?  kiss off, geek. what I said is true...
27450690        " Please do not vandalize pages, as you did wi...
54037174        " ""Points of interest"" I removed the ""point...
77493077        Asking some his nationality is a Racial offenc...
79357270        The reader here is not going by my say so for ...
82428052                Fried chickens Is dat sum fried chickens?
87311443        Why can you put English for example on some pl...
114749757       Guy Fawkes im a resident in bridgwater and i g...
138560519       as far as nicknames go this article is embarra...
139353149       Woodland MeadowsGood to hear that you correcte...
150009866       "Well I just finished a good bit of editing.  ...
152276337       Discussion should take place on the article ta...
153331729       Uh oh, you called my bluff. I am intimidated b...
153600803       "We should also contact the living descendants...
154186883       " May 2008 (UTC)Notability of Your New Heart A...
1618941

In [ ]:
## Here is a list of lists, each containing the tokens in each chat
sentences = 

## Word2vec with Gensim

In [ ]:
## For this, we need to first create a list of lists called 'sentences' that represents the words in 
## each of the sentences / chats
model = Word2Vec(sentences, min_count=3)
print (model)  #summarize the trained model

In [ ]:
## Persist the model and continue training if needed
model.save('W2Vmodel1')
model = Word2Vec.load('W2Vmodel1')  # you can continue training with the loaded model!

In [ ]:
model.wv['computer'] # numpy vector of a word

In [ ]:
model.wv.most_similar(positive=['woman', 'king'], negative=['man'])

# With cosine multiplication as in Yoav Goldberg's 2014 paper
model.wv.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])
